In [2]:
import pandas as pd
from api.core.mongo import db
from langchain_text_splitters import RecursiveJsonSplitter
from nomic import embed
import logging

logging.basicConfig(level=logging.DEBUG)

C:\Users\holmt\PycharmProjects\sunhacks-2024\venv\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [3]:
rows = pd.DataFrame(list(db["products"].find({
    "product_type": {"$in": ["ski", "phone"]},
}))).drop(columns=["_id"]).to_dict(orient="records")

DEBUG:pymongo.serverSelection:{"message": "Server selection started", "selector": "Primary()", "operation": "find", "topologyDescription": "<TopologyDescription id: 66f97ecd75bdd444ddb20511, topology_type: ReplicaSetWithPrimary, servers: [<ServerDescription ('cluster0-shard-00-00.6le3g.mongodb.net', 27017) server_type: RSSecondary, rtt: 0.04699999999957072>, <ServerDescription ('cluster0-shard-00-01.6le3g.mongodb.net', 27017) server_type: RSPrimary, rtt: 0.04699999999957072>, <ServerDescription ('cluster0-shard-00-02.6le3g.mongodb.net', 27017) server_type: RSSecondary, rtt: 0.04699999999957072>]>", "clientId": {"$oid": "66f97ecd75bdd444ddb20511"}}
DEBUG:pymongo.serverSelection:{"message": "Server selection succeeded", "selector": "Primary()", "operation": "find", "topologyDescription": "<TopologyDescription id: 66f97ecd75bdd444ddb20511, topology_type: ReplicaSetWithPrimary, servers: [<ServerDescription ('cluster0-shard-00-00.6le3g.mongodb.net', 27017) server_type: RSSecondary, rtt: 0.0

In [4]:
splitter = RecursiveJsonSplitter(max_chunk_size=300)
documents = splitter.create_documents(rows)

INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: cpu
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: nomic-ai/nomic-embed-text-v1
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): huggingface.co:443
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /nomic-ai/nomic-embed-text-v1/resolve/main/modules.json HTTP/11" 200 0
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /nomic-ai/nomic-embed-text-v1/resolve/main/config_sentence_transformers.json HTTP/11" 200 0
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /nomic-ai/nomic-embed-text-v1/resolve/main/README.md HTTP/11" 200 0
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /nomic-ai/nomic-embed-text-v1/resolve/main/modules.json HTTP/11" 200 0
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /nomic-ai/nomic-embed-text-v1/resolve/main/sentence_bert_config.json HTTP/11" 200 0
DEBUG:urllib3.connectionpool:htt

In [ ]:
# Store the data as vector embeddings in Atlas

embeddings = embed.text([document.page_content for document in documents], inference_mode="local")
db["embeddings"].insert_many(embeddings["embeddings"])
        

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): gpt4all.io:443
DEBUG:urllib3.connectionpool:https://gpt4all.io:443 "GET /models/models3.json HTTP/11" 301 167
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): raw.githubusercontent.com:443
DEBUG:urllib3.connectionpool:https://raw.githubusercontent.com:443 "GET /nomic-ai/gpt4all/main/gpt4all-chat/metadata/models3.json HTTP/11" 200 4020
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): gpt4all.io:443
DEBUG:urllib3.connectionpool:https://gpt4all.io:443 "GET /models/gguf/nomic-embed-text-v1.5.f16.gguf HTTP/11" 200 274290560
Downloading:  97%|█████████▋| 266M/274M [04:29<00:39, 203kiB/s]  
Download interrupted, resuming from byte position 268435456
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): gpt4all.io:443
DEBUG:urllib3.connectionpool:https://gpt4all.io:443 "GET /models/gguf/nomic-embed-text-v1.5.f16.gguf HTTP/11" 206 5855104
Downloading: 100%|██████████| 274M/274M [04:30<00:00, 